In [2]:
import json
import csv
import os
from transformers import AutoTokenizer, AutoModelForCausalLM


In [9]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.2-1B.
401 Client Error. (Request ID: Root=1-670bd01e-418aad3e128cbf04355567e2;00d80d34-e8ed-47a6-8c90-d803b6e37493)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B is restricted. You must have access to it and be authenticated to access it. Please log in.

In [8]:


# Define the maximum length for the generated code
max_length = 200 # Adjust as needed

# Function to generate code using the specified model
def generate_code(prompt, model_name="Salesforce/codegen-350M-mono"):
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate code
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        pad_token_id=tokenizer.eos_token_id,
        max_length=max_length 
    )

    # Decode the generated code
    generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_code

# Function to automatically locate the dataset
def locate_dataset(dataset_name="concode/test.json"):
    current_dir = os.getcwd()
    dataset_path = os.path.join(current_dir, dataset_name)

    if os.path.exists(dataset_path):
        return dataset_path
    else:
        raise FileNotFoundError(f"Dataset '{dataset_name}' not found in the current directory: {current_dir}")

# Function to read the Concode dataset (handling multiple JSON objects)
def load_concode_dataset(dataset_path):
    data = []
    with open(dataset_path, "r") as file:
        for line in file:
            try:
                data.append(json.loads(line.strip()))  # Load each line as JSON
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON on line: {line}")
                continue
    return data

# Function to clean the natural language description by removing separators (if needed)
def clean_nl(nl):
    return nl.replace("concode_field_sep", "|").replace("concode_elem_sep", "->")

# Function to save results to CSV
def save_to_csv(results, output_csv="C350_output.csv"):
    keys = results[0].keys()
    with open(output_csv, "w", newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=keys)
        writer.writeheader()
        writer.writerows(results)

# Function to save results to JSON
def save_to_json(results, output_json="C350_output.json"):
    with open(output_json, "w") as jsonfile:
        json.dump(results, jsonfile, indent=4)

# Main code to load dataset, generate code and save output
if __name__ == "__main__":
    try:
        # Automatically locate the dataset in the current directory
        dataset_path = locate_dataset()

        # Load dataset
        dataset = load_concode_dataset(dataset_path)

        # Limit to 10 data items
        dataset_sample = dataset[:10]

        # List to store results
        results = []

        # Iterate through the dataset sample and generate code for each example
        for i, example in enumerate(dataset_sample):
            natural_language_desc = clean_nl(example["nl"])  # Clean natural language description
            java_code = generate_code(natural_language_desc, model_name="Salesforce/codegen-2.7B")

            # Store the result
            result = {
                "Example": i + 1,
                "Description": natural_language_desc,  # Uncomment if needed
                "Generated Java Code": java_code
            }
            results.append(result)

            print(f"Example {i + 1}:")
            print(f"Description: {natural_language_desc}")  # Uncomment if needed
            print(f"Generated Java Code:\n{java_code}")
            print("-" * 50)

        # Save the results to CSV and JSON
        save_to_csv(results)
        save_to_json(results)

        print("Results saved to CSV and JSON files.")

    except FileNotFoundError as e:
        print(e)


OSError: Salesforce/codegen-2.7B is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`